# Time series predictor

In [9]:
import numpy as np
import time
from datetime import date
import datetime
from datetime import timedelta  
import csv
import holidays # for importing the public holidays
import re
import torch
from torch import nn
from src.utils import *
from src.data_miner import DataMiner

## Parameters

In [2]:
num_features = 5
min_hour = 21 # Minimum hour for sleep detection
max_hour = 5
train_window = 3 # Sequence length
local_holidays = holidays.Italy(prov='BO') # Get the holidays in Bologna, Italy :)
train_episodes = 500
batch_size = 3

In [3]:
# Variables
data_dir = "data"
dataset = "data/LastSeenDataset.csv"

- Feature extraction: we first extract the features given the time series data of Telegram accesses.
- Supposition: last Telegram access in very similar to the time the person goes to sleep

## Feature engineering
Possible features to extract: 
1. Last seen time (arguably the most important)
2. Wake up time
3. Number of Telegram accesses during the previous day
4. Day of the week
5. Public holiday presence in the following day (using the holidays library)
6. (time spent on Telegram)


In [4]:
with open(dataset, newline='') as csvfile:
    date_list = list(csv.reader(csvfile))

date_list = convert_to_dates(date_list)

'''Test data: search calendar for local holidays'''
print("First day is holiday: ", date_list[0][0] in local_holidays)

First day is holiday:  False


In [5]:
data_tensor =  DataMiner(date_list).to_tensor(verbose=False)
print(data_tensor)

tensor([[0.6002, 0.5434, 0.4465, 0.5033, 0.4888, 0.5380, 0.5200, 0.6680, 0.1981,
         0.5418, 0.5891, 0.5230, 0.5878, 0.2870, 0.1545, 0.3483, 0.1007, 0.6694,
         0.5091, 0.4906, 0.6093, 0.6412, 0.8530, 0.3883, 0.5664, 0.7656],
        [0.6667, 0.5991, 0.6653, 0.6445, 0.7801, 0.6894, 0.6742, 0.6647, 1.0048,
         0.6278, 0.7105, 0.6988, 0.6384, 0.8407, 0.9146, 0.7862, 1.1783, 0.4033,
         0.6723, 0.6988, 0.6034, 0.5998, 0.5354, 0.7193, 0.7006, 0.5691],
        [1.0000, 0.0000, 0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000, 0.0000,
         0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000, 0.0000, 0.1667, 0.3333,
         0.5000, 0.6667, 0.8333, 1.0000, 0.0000, 0.1667, 0.3333, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1400, 0.1200, 0.0600, 0.0500, 0.0700, 0.1400,

## Data augmentation

Given that the training data is not much, we can insert some noise to augment it; this will also make the model less prone to overfitting

In [6]:
# Data augmentation


# We use the "last 3" trend
# Credits: https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
'''The sequence on which we have a prediction is the last train_window days'''
X, y = create_inout_sequences(data_tensor, train_window)
X = X.transpose(0, 2, 1)

## Model
- Time series data, so possible idea(s):
    - LSTM

In [7]:
from src.models import RNN

In [16]:
n_features = num_features # this is number of parallel inputs
n_timesteps = train_window # this is number of timesteps

# convert dataset into input/output

# create NN
model = RNN(n_features, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

print(X[2])
print(y[1])

[[0.44649306 0.66525465 0.16666667 0.         0.06      ]
 [0.50329864 0.64453703 0.33333334 0.         0.05      ]
 [0.48875    0.7801157  0.5        0.         0.07      ]]
[0.48875]


In [17]:
model.train()

# Training loop
for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net.init_hidden(x_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_net(x_batch) 
        loss = criterion(output, y_batch)  
#         print('PREDICTED:\n', output); print('REAL:\n', y_batch)
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad()
        #loss_list.append(loss.item())
    if t%10:
        print(('Step: {:4}   |   Loss: {:.6f} ').format(t, loss.item()))

ModuleAttributeError: 'RNN' object has no attribute 'init_hidden'

In [ ]:
b = 1
inpt = X[-b-batch_size:-b,:,:]
target = y[b:b+batch_size]
x_batch = torch.tensor(inpt,dtype=torch.float32)    
y_batch = torch.tensor(target,dtype=torch.float32)

#    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
#    lstm_out.contiguous().view(x_batch.size(0),-1)
mv_net.init_hidden(x_batch.size(0))

print(x_batch.size())
output = mv_net(torch.tensor(X[-1-batch_size:-1,:,:],dtype=torch.float32))[0]

with torch.no_grad():
    print(mv_net(torch.tensor(X[-1-batch_size:-1,:,:],dtype=torch.float32))[0])

In [ ]:
with torch.no_grad():
    print('Predicted:', mv_net(torch.tensor(X[-1-batch_size:-1,:,:],dtype=torch.float32))[0])
    print('Real:', y[-1])

## Saving the time

We save the predicted time to send the message in a file, so that the Daemon can handle it

In [ ]:
now = datetime.datetime.now()
# with torch.no_grad():
p = mv_net.forward(torch.tensor(X[-batch_size-1:-1:,:],dtype=torch.float32))[0].detach().numpy()
p_sec = int(p[0]*(max_hour+24-min_hour)*3600)
prediction = now.replace(hour=min_hour, minute=0, second=0) + timedelta(seconds=p_sec)
print('Expected time to go to sleep: ', prediction.strftime("%Y-%m-%d %H:%M:%S"))


'''Write the value on a text file to be read by the Daemon'''
with open ('prediction.txt','w') as z:
    z.write(prediction.strftime("%Y-%m-%d %H:%M:%S\n"))
z.close()

with open ('data/prediction_list.txt','a') as z:
    z.write(prediction.strftime("%Y-%m-%d %H:%M:%S\n"))
z.close()